In [1]:
# Dependencies
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter("ignore")
import psycopg2
from psycopg2 import sql

from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from imblearn.over_sampling import RandomOverSampler

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

# READ Data from Database
#### Here we are going to read data from the database and creating a Dataframe

In [2]:
# Define your PostgreSQL connection parameters
db_params = {
    "host": "localhost",
    "database": "credit_card_data",
    "user": "postgres",
    "password": ""
}

cols = ["Transaction ID", "Date", "Day of Week", "Time", "Type of Card", "Entry Mode",
        "Amount", "Type of Transaction", "Merchant Group", "Country of Transaction",
        "Shipping Address", "Country of Residence", "Gender", "Age", "Bank", "Fraud"]
column_list = [
    ('transaction_id', 'varchar(9)'),
    ('date', 'varchar(9)'),
    ('day_of_week', 'varchar(9)'),
    ('time', 'int4'),
    ('type_of_card', 'varchar(10)'),
    ('entry_mode', 'varchar(3)'),
    ('amount', 'varchar'),
    ('type_of_transaction', 'varchar(6)'),
    ('merchant_group', 'varchar(13)'),
    ('country_of_transaction', 'varchar(14)'),
    ('shipping_address', 'varchar(14)'),
    ('country_of_residence', 'varchar(14)'),
    ('gender', 'varchar(1)'),
    ('age', 'numeric(4,1)'),
    ('bank', 'varchar(8)'),
    ('fraud', 'bit(1)')
]

# Establish a connection to the database
connection = psycopg2.connect(**db_params)
print("Connected Successfully")

# Create a cursor object
cursor = connection.cursor()

# Generate a SQL query to select the desired columns
select_columns = ", ".join([f'"{col_name}"' for col_name, _ in column_list])
query = f"SELECT {select_columns} FROM public.credit_card_data"

# Execute the query
cursor.execute(query)

# Fetch all rows from the result set
rows = cursor.fetchall()

# Create a DataFrame from the fetched data
df = pd.DataFrame(rows, columns=[col_name for col_name in cols])

# Close the cursor and the database connection
cursor.close()
connection.close()
print("DataFrame Generated Successfully")



Connected Successfully
DataFrame Generated Successfully


In [3]:
df.head()

,Transaction ID,Date,Day of Week,Time,Type of Card,Entry Mode,Amount,Type of Transaction,Merchant Group,Country of Transaction,Shipping Address,Country of Residence,Gender,Age,Bank,Fraud
0,#3577 209,14-Oct-20,Wednesday,19,Visa,Tap,Â£5,POS,Entertainment,United Kingdom,United Kingdom,United Kingdom,M,25.2,RBS,0
1,#3039 221,14-Oct-20,Wednesday,17,MasterCard,PIN,Â£288,POS,Services,USA,USA,USA,F,49.6,Lloyds,0
2,#2694 780,14-Oct-20,Wednesday,14,Visa,Tap,Â£5,POS,Restaurant,India,India,India,F,42.2,Barclays,0
3,#2640 960,13-Oct-20,Tuesday,14,Visa,Tap,Â£28,POS,Entertainment,United Kingdom,India,United Kingdom,F,51.0,Barclays,0
4,#2771 031,13-Oct-20,Tuesday,23,Visa,CVC,Â£91,Online,Electronics,USA,USA,United Kingdom,M,38.0,Halifax,1


In [4]:
# Reading and importing data from postgres database table
CreditCardData_df = df
CreditCardData_df.head()

,Transaction ID,Date,Day of Week,Time,Type of Card,Entry Mode,Amount,Type of Transaction,Merchant Group,Country of Transaction,Shipping Address,Country of Residence,Gender,Age,Bank,Fraud
0,#3577 209,14-Oct-20,Wednesday,19,Visa,Tap,Â£5,POS,Entertainment,United Kingdom,United Kingdom,United Kingdom,M,25.2,RBS,0
1,#3039 221,14-Oct-20,Wednesday,17,MasterCard,PIN,Â£288,POS,Services,USA,USA,USA,F,49.6,Lloyds,0
2,#2694 780,14-Oct-20,Wednesday,14,Visa,Tap,Â£5,POS,Restaurant,India,India,India,F,42.2,Barclays,0
3,#2640 960,13-Oct-20,Tuesday,14,Visa,Tap,Â£28,POS,Entertainment,United Kingdom,India,United Kingdom,F,51.0,Barclays,0
4,#2771 031,13-Oct-20,Tuesday,23,Visa,CVC,Â£91,Online,Electronics,USA,USA,United Kingdom,M,38.0,Halifax,1


---------------

## Cleaning and Preparation

--------------